In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day28-column-transformer/covid_toy.csv")
df.sample(5)

,age,gender,fever,cough,city,has_covid
43,22,Female,99.0,Mild,Bangalore,Yes
95,12,Female,104.0,Mild,Bangalore,No
23,80,Female,98.0,Mild,Delhi,Yes
27,33,Female,102.0,Strong,Delhi,No
82,24,Male,98.0,Mild,Kolkata,Yes


In [3]:
df.shape

(100, 6)

In [4]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [5]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

age,fever - numerical

gender,city - nominal

cough - ordinal

In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

fever - simpleimputer

gender,city - onehotencoder

cough - ordinalencoder

has_covid - labelencoder

### Train Test Split

In [9]:
from sklearn.model_selection import train_test_split

In [11]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df.has_covid,test_size=0.2)

In [13]:
x_train

,age,gender,fever,cough,city
41,82,Male,NaN,Mild,Kolkata
89,46,Male,103.0,Strong,Bangalore
62,56,Female,104.0,Strong,Bangalore
94,79,Male,NaN,Strong,Kolkata
49,44,Male,104.0,Mild,Mumbai
...,...,...,...,...,...
70,68,Female,101.0,Strong,Delhi
60,24,Female,102.0,Strong,Bangalore
86,25,Male,104.0,Mild,Bangalore
43,22,Female,99.0,Mild,Bangalore


#### Traditional Way of Encoding

In [14]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [15]:
#adding simple imputer to fever
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.fit_transform(x_test[['fever']])

In [16]:
#ordinalencoder for cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.fit_transform(x_test[['cough']])

In [17]:
#onehotencoder for gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

In [18]:
#Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

Finally Combining all the columns

In [19]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

In [22]:
x_train_transformed.shape

(80, 7)

### Column Transformer

In [23]:
from sklearn.compose import ColumnTransformer

In [24]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',si,['fever']),
    ('tnf2',oe,['cough']),
    ('tnf3',ohe,['gender','city'])
],remainder='passthrough')

In [26]:
transformer.fit_transform(x_train).shape

(80, 7)